In [1]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

/home/ubuntu/Qwen-Image-Edit-Angles


In [2]:

from qwenimage.debug import clear_cuda_memory, print_gpu_memory
from qwenimage.experiment import ExperimentConfig
from qwenimage.experiments.experiments_qwen import PipeInputs, Qwen_AoT, QwenBaseExperiment, ExperimentRegistry

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
TMA benchmarks will be running without grid constant TMA descriptor.
2025-11-13 14:36:44.418437: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the envi

In [3]:
name = "qwen_base"

experiment = ExperimentRegistry.get(name)(
    config=ExperimentConfig(
        name=name,
    ), 
)
experiment.load()


720 input combinations


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  6.24it/s]
Expected types for transformer: (<class 'diffusers.models.transformers.transformer_qwenimage.QwenImageTransformer2DModel'>,), got <class 'qwenimage.models.transformer_qwenimage.QwenImageTransformer2DModel'>.


Time taken by QwenBaseExperiment.load: 21.346381488998304 seconds


In [4]:
experiment.pipe.transformer

QwenImageTransformer2DModel(
  (pos_embed): QwenEmbedRope()
  (time_text_embed): QwenTimestepProjEmbeddings(
    (time_proj): Timesteps()
    (timestep_embedder): TimestepEmbedding(
      (linear_1): Linear(in_features=256, out_features=3072, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=3072, out_features=3072, bias=True)
    )
  )
  (txt_norm): RMSNorm()
  (img_in): Linear(in_features=64, out_features=3072, bias=True)
  (txt_in): Linear(in_features=3584, out_features=3072, bias=True)
  (transformer_blocks): ModuleList(
    (0-59): 60 x QwenImageTransformerBlock(
      (img_mod): Sequential(
        (0): SiLU()
        (1): Linear(in_features=3072, out_features=18432, bias=True)
      )
      (img_norm1): LayerNorm((3072,), eps=1e-06, elementwise_affine=False)
      (attn): Attention(
        (norm_q): RMSNorm()
        (norm_k): RMSNorm()
        (to_q): Linear(in_features=3072, out_features=3072, bias=True)
        (to_k): Linear(in_features=3072, out_features=

In [5]:
experiment.pipe.transformer.fuse_qkv_projections()

In [9]:
experiment.pipe.transformer.transformer_blocks[0].attn.processor

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.processor

In [10]:
experiment.pipe.transformer.transformer_blocks[0].attn.processor.fuse_projections()

AttributeError: 'QwenDoubleStreamAttnProcessor2_0' object has no attribute 'fuse_projections'

In [11]:
experiment.pipe.transformer.transformer_blocks[0].attn.fuse_projections()

In [12]:
experiment.pipe.transformer.transformer_blocks[0].attn

Attention(
  (norm_q): RMSNorm()
  (norm_k): RMSNorm()
  (to_q): Linear(in_features=3072, out_features=3072, bias=True)
  (to_k): Linear(in_features=3072, out_features=3072, bias=True)
  (to_v): Linear(in_features=3072, out_features=3072, bias=True)
  (add_k_proj): Linear(in_features=3072, out_features=3072, bias=True)
  (add_v_proj): Linear(in_features=3072, out_features=3072, bias=True)
  (add_q_proj): Linear(in_features=3072, out_features=3072, bias=True)
  (to_out): ModuleList(
    (0): Linear(in_features=3072, out_features=3072, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
  (to_add_out): Linear(in_features=3072, out_features=3072, bias=True)
  (norm_added_q): RMSNorm()
  (norm_added_k): RMSNorm()
  (to_qkv): Linear(in_features=3072, out_features=9216, bias=True)
  (to_added_qkv): Linear(in_features=3072, out_features=9216, bias=True)
)

In [18]:
experiment.pipe.transformer.transformer_blocks[0].attn.to_qkv.weight

Parameter containing:
tensor([[-0.0354, -0.0508,  0.0098,  ..., -0.0466,  0.0349, -0.0154],
        [ 0.0036,  0.1016,  0.0059,  ..., -0.2812,  0.0466,  0.0233],
        [ 0.0041,  0.0253, -0.0157,  ..., -0.0137,  0.0294,  0.0137],
        ...,
        [-0.0354, -0.0393, -0.0237,  ...,  0.0352,  0.0315,  0.0058],
        [ 0.0214, -0.0430,  0.0119,  ...,  0.0547,  0.0352, -0.0117],
        [-0.0315, -0.0703, -0.0292,  ...,  0.0859, -0.0270, -0.0097]],
       device='cuda:0', dtype=torch.bfloat16, requires_grad=True)

In [17]:
experiment.pipe.transformer.transformer_blocks[0].attn.to_q.weight

Parameter containing:
tensor([[-0.0354, -0.0508,  0.0098,  ..., -0.0466,  0.0349, -0.0154],
        [ 0.0036,  0.1016,  0.0059,  ..., -0.2812,  0.0466,  0.0233],
        [ 0.0041,  0.0253, -0.0157,  ..., -0.0137,  0.0294,  0.0137],
        ...,
        [ 0.0258,  0.0508,  0.0137,  ..., -0.0430,  0.0197, -0.0007],
        [-0.0349,  0.0058,  0.0195,  ..., -0.0255,  0.0100,  0.0289],
        [ 0.0312, -0.0703, -0.0177,  ...,  0.0198, -0.0233, -0.0060]],
       device='cuda:0', dtype=torch.bfloat16)

In [21]:
experiment.pipe.transformer.transformer_blocks[0].attn.add_q_proj.weight

Parameter containing:
tensor([[-0.1089,  0.1406,  0.0310,  ...,  0.0623, -0.1016,  0.0859],
        [ 0.0391,  0.0002,  0.0312,  ...,  0.0505,  0.0208, -0.0549],
        [ 0.0055, -0.0703, -0.0471,  ..., -0.0171, -0.0874,  0.0625],
        ...,
        [-0.0386,  0.0703, -0.0116,  ..., -0.0004, -0.0015,  0.0037],
        [-0.0869, -0.0229,  0.0586,  ..., -0.0092,  0.1875, -0.0231],
        [-0.0182,  0.0432,  0.0019,  ..., -0.0152, -0.1250,  0.0471]],
       device='cuda:0', dtype=torch.bfloat16)

In [22]:
experiment.pipe.transformer.transformer_blocks[0].attn.to_added_qkv.weight

Parameter containing:
tensor([[-1.0889e-01,  1.4062e-01,  3.1006e-02,  ...,  6.2256e-02,
         -1.0156e-01,  8.5938e-02],
        [ 3.9062e-02,  2.1744e-04,  3.1250e-02,  ...,  5.0537e-02,
          2.0752e-02, -5.4932e-02],
        [ 5.5237e-03, -7.0312e-02, -4.7119e-02,  ..., -1.7090e-02,
         -8.7402e-02,  6.2500e-02],
        ...,
        [ 5.9509e-03,  3.9062e-02,  1.3550e-02,  ...,  2.0905e-03,
          1.3611e-02,  3.8452e-03],
        [-3.9062e-02, -7.0312e-02, -3.7384e-03,  ...,  1.8158e-03,
          2.1875e-01,  5.4688e-02],
        [-8.5938e-02, -1.3611e-02,  3.1128e-02,  ...,  2.5391e-02,
         -1.0938e-01,  1.7700e-02]], device='cuda:0', dtype=torch.bfloat16,
       requires_grad=True)

In [23]:
experiment.pipe.scheduler

FlowMatchEulerDiscreteScheduler {
  "_class_name": "FlowMatchEulerDiscreteScheduler",
  "_diffusers_version": "0.36.0.dev0",
  "base_image_seq_len": 256,
  "base_shift": 0.5,
  "invert_sigmas": false,
  "max_image_seq_len": 8192,
  "max_shift": 0.9,
  "num_train_timesteps": 1000,
  "shift": 1.0,
  "shift_terminal": 0.02,
  "stochastic_sampling": false,
  "time_shift_type": "exponential",
  "use_beta_sigmas": false,
  "use_dynamic_shifting": true,
  "use_exponential_sigmas": false,
  "use_karras_sigmas": false
}

In [ ]:

import math
from diffusers.schedulers.scheduling_flow_match_euler_discrete import FlowMatchEulerDiscreteScheduler
import torch

from qwenimage.models.pipeline_qwenimage_edit_plus import QwenImageEditPlusPipeline
from qwenimage.models.transformer_qwenimage import QwenImageTransformer2DModel

# Scheduler configuration for Lightning
scheduler_config = {
    "base_image_seq_len": 256,
    "base_shift": math.log(3),  # We use shift=3 in distillation
    "invert_sigmas": False,
    "max_image_seq_len": 8192,
    "max_shift": math.log(3),  # We use shift=3 in distillation
    "num_train_timesteps": 1000,
    "shift": 1.0,
    "shift_terminal": None,  # set shift_terminal to None
    "stochastic_sampling": False,
    "time_shift_type": "exponential",
    "use_beta_sigmas": False,
    "use_dynamic_shifting": True,
    "use_exponential_sigmas": False,
    "use_karras_sigmas": False,
}

# Initialize scheduler with Lightning config

scheduler = FlowMatchEulerDiscreteScheduler.from_config(scheduler_config) # TODO: check scheduler sync issue mentioned by https://pytorch.org/blog/presenting-flux-fast-making-flux-go-brrr-on-h100s/

dtype = torch.bfloat16
device = "cuda" if torch.cuda.is_available() else "cpu"


pipe = QwenImageEditPlusPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit-2509", 
    transformer=QwenImageTransformer2DModel.from_pretrained( # TODO: remove this if using lightning
        "linoyts/Qwen-Image-Edit-Rapid-AIO", 
        subfolder='transformer',
        torch_dtype=dtype,
        device_map='cuda'),
    scheduler=scheduler,
    torch_dtype=dtype,
).to(device)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-11-13 15:17:10.085053: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 15:17:10.099287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E

In [6]:
import torch

e = torch.rand((1, 253, 3584), dtype=torch.bfloat16)
e.numel() * e.element_size() / (1024 ** 2)

1.7294921875

In [ ]:
mvae_params = sum(p.numel() for p in flux.blend_model.mvae.parameters())
model_size_mb = sum(p.numel() * p.element_size() for p in flux.blend_model.mvae.parameters()) / (1024 ** 2)
print(f"mvae parameter count: {mvae_params:,}")
print(f"mvae model size: {model_size_mb:.2f} MB")

transformer_params = sum(p.numel() for p in flux.transformer.parameters())
model_size_mb = sum(p.numel() * p.element_size() for p in flux.transformer.parameters()) / (1024 ** 2)
print(f"flux.transformer parameter count: {transformer_params:,}")
print(f"flux.transformer model size: {model_size_mb:.2f} MB")

vae_params = sum(p.numel() for p in flux.vae.parameters())
model_size_mb = sum(p.numel() * p.element_size() for p in flux.vae.parameters()) / (1024 ** 2)
print(f"flux.vae parameter count: {vae_params:,}")
print(f"flux.vae model size: {model_size_mb:.2f} MB")

print(f"\nParameter comparisons:")
print(f"mvae vs transformer: {mvae_params / transformer_params * 100:.3f}%")
print(f"mvae vs vae: {mvae_params / vae_params * 100:.3f}%")
